### 2D CNN Model

In [1]:
from __future__ import print_function
from hyperas import optim
from hyperas.distributions import uniform
from hyperopt import Trials, STATUS_OK, tpe
# https://github.com/maxpumperla/hyperas

/Users/Nahel/anaconda3/envs/NN/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
from keras.optimizers import Adam

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Conv2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
import h5py

import pandas as pd
import numpy as np

import keras
from keras.models import Model, Sequential
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.callbacks import EarlyStopping

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def Data_Prep():
    nb_classes = 1
    
    data = pd.read_csv('./2D_120.csv')

    target = data.AGE.copy(deep=True)
    data = data.drop(['Unnamed: 0','AGE'],axis=1)

    X_train, X_test, y_train, y_test = train_test_split(
    data,target,test_size=0.2,random_state=42)

    mm = MinMaxScaler()
    X_train = mm.fit_transform(X_train)
    X_test  = mm.transform(X_test)

    X_train = X_train.reshape(X_train.shape[0], 256, 150, 1)
    X_test  = X_test.reshape(X_test.shape[0], 256, 150, 1)
    return X_train, X_test, y_train, y_test

In [ ]:
def model(X_train, Y_train, X_test, Y_test):

    nb_epoch = 15
    
    nb_classes = 1

    # input image dimensions
    img_rows, img_cols = 256, 150
    img_channels = 1

    model = Sequential()

    model.add(Conv2D({{choice([90, 180])}}, (5,5), padding='same',
                            input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (4,4), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout({{uniform(0, 1)}}))

    model.add(Flatten())
    model.add(Dense({{choice([1024, 2048])}}))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(nb_classes))
    model.add(Activation('relu'))

    adam = Adam()
    model.compile(loss='mean_absolute_error',
                  optimizer=adam, 
                  metrics=['mean_squared_error'])

    model.fit(X_train, y_train,
            batch_size=128,
            epochs=nb_epoch,
            validation_data=(X_test, y_test))

    score, acc = model.evaluate(X_test, y_test, verbose=1)

    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [ ]:
X_train, X_test, y_train, y_test = Data_Prep()

In [ ]:
best_run, best_model, space = optim.minimize(model=model,
                                          data=Data_Prep,
                                          algo=tpe.suggest,
                                          max_evals=2,
                                          trials=Trials(),
                                          #eval_space=True,  
                                          #return_space=True,
                                          notebook_name='2D_CNN_Model_Opt'
                                            )

In [ ]:
sns.regplot(best_model.predict(X_test), y_test)
#sns.plt.show()